In [1]:
# Basics
import numpy as np
import pandas as pd
import os
import pickle
import h5py

# Keras
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard

/Users/anders1991/miniconda3/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load data partitions

In [2]:
with open("./data/partition.pkl", 'rb') as f:
    partition = pickle.load(f)

# Load labels

In [3]:
target = pd.read_csv("./data/ERU_Scores_Ids_5-Scans_Validity-0_VisuallyScored.csv")
labels = target.set_index("StId").to_dict()["ERU.M2"]

Rescale labels to fit model softmax output

In [4]:
label_converter = {0: 0.0, 1: 0.0, 2: 0.03, 3: 0.155, 4: 0.38, 5: 0.63, 6: 0.88} # combine 0+1 as 0 = no emph in scan, 1 = no emph in region
labels = {key: label_converter[val] for key, val in labels.items()}

# Calculate class weights

To handle imbalanced classes during training.

In [5]:
from sklearn.utils import class_weight

In [6]:
# weights only based on training data
train_labels = [label for key, label in labels.items() if key in partition["train"]]

In [7]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(train_labels), 
                                                  y=train_labels)

# Set hyperparameters

In [8]:
timesteps = 1
batch_size = 4
learn_rate = 1e-4
max_epochs = 50
name = "single"
downsample = 2

# Import data generator

In [9]:
from data_gen import DataGenerator

In [10]:
trainGen = DataGenerator("single_train", batch_size=batch_size, timesteps=timesteps, 
                         channels=1, dim_x=142, dim_y=322, dim_z=262, shuffle=True)
validGen = DataGenerator("single_valid", batch_size=batch_size, timesteps=timesteps, 
                         channels=1, dim_x=142, dim_y=322, dim_z=262, shuffle=False)

trainGen = trainGen.generate(labels, partition["train"])
validGen = validGen.generate(labels, partition["valid"])

# Import model

In [13]:
from unet import tdist_unet

In [14]:
model = tdist_unet(timesteps=timesteps, downsample=downsample)

In [15]:
model.compile(optimizer=Adam(lr = learn_rate), 
              loss='mean_absolute_error', 
              metrics=['accuracy', 'mae', 'mse'])

# Setup training session

In [24]:
callbacks_list = []

Save model weights each epoch if best val acc so far

In [17]:
directory = "../models/single/"
if not os.path.exists(directory):
    os.makedirs(directory)
modeldir = directory + "epoch_{epoch:02d}-valacc_{val_acc:.2f}.hdf5"

checkpoint = ModelCheckpoint(modeldir, monitor='val_acc', save_weights_only=False, save_best_only=True, mode='max', verbose=1)
callbacks_list.append(checkpoint)

# Load model weights using:
# model.load_weights(modeldir)

Add TensorBoard callback

In [18]:
tensorboard = TensorBoard(log_dir='../graph', histogram_freq=1, write_graph=True, write_images=True)
callbacks_list.append(tensorboard)

# Access tensorboard using:
# tensorboard --logdir path_to_current_dir/graph 

In [21]:
from TBCallbacks import TrainValTensorBoard

trainvalTB = TrainValTensorBoard(log_dir="./logs", histogram_freq=1, write_graph=True, write_images=True) # custom TB writer object
callbacks_list.append(trainvalTB) # add tensorboard logging


Train model

In [18]:
hist = model.fit_generator(generator = trainGen,
                           steps_per_epoch = len(partition["train"])//batch_size,
                           validation_data = validGen,
                           validation_steps = len(partition["valid"])//batch_size,
                           class_weight = class_weights,
                           epochs = max_epochs,
                           callbacks=callbacks_list)

Epoch 1/1
